In [ ]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import scipy.signal as sg
from utils import utils_gn, utils_dgrd, utils_models
import importlib
importlib.reload(utils_gn)
importlib.reload(utils_models)
importlib.reload(utils_dgrd)
warnings.filterwarnings("ignore")   
#plt.style.use('science')

In [ ]:
# Load the test raw data
test_raw = utils_gn.read_data('test_1238.pkl')

In [ ]:
# Load saved models and transformations
cycle_model, cycle_trans = utils_gn.read_data('sig_cycles.pkl', folder='models'), utils_gn.read_data('sig_cycles_trans.pkl', folder='models')
cap_ir_model, cap_ir_trans = utils_gn.read_data('sig_capacity_ir.pkl', folder='models'), utils_gn.read_data('sig_capacity_ir_trans.pkl', folder='models')

In [ ]:
# Make predictions 
cycle_pred = cycle_model.predict(cycle_trans.transform(test_raw, sig_level=2, multi_cycle=False))
capir_pred = cap_ir_model.predict(cap_ir_trans.transform(test_raw, sig_level=2, multi_cycle=False))

In [ ]:
# Load prediction intervals
cycle_pred_interval = utils_gn.read_data('sig_cycles_pred_interval.pkl', folder='models')
capir_pred_interval = utils_gn.read_data('sig_capir_pred_interval.pkl', folder='models')

In [ ]:
# Create dataframe of predicted values and their prediction intervals
target_list = ['k-o', 'k-p', 'e-o', 'e-p', 'EOL', 'Qatk-o', 'Qatk-p', 'IRate-o', 'IRate-p', 'IRatEOL']

prediction_df = pd.DataFrame(index=test_raw.keys(), columns=target_list)

for i, cell in enumerate(test_raw.keys()):
    prediction_df.loc[cell, ['k-o', 'k-p', 'e-o', 'e-p', 'EOL']] = cycle_pred[i]
    prediction_df.loc[cell, ['Qatk-o', 'Qatk-p', 'IRate-o', 'IRate-p', 'IRatEOL']] = capir_pred[i]

for i, target in enumerate(target_list[:5]):
    prediction_df[f'{target} CI'] = cycle_pred_interval[i]

for i, target in enumerate(target_list[5:]):
    prediction_df[f'{target} CI'] = capir_pred_interval[i]

columns_rearranged = []
for target in target_list:
    columns_rearranged.extend((target, f'{target} CI'))
prediction_df = prediction_df[columns_rearranged]
prediction_df.head()

In [ ]:
sample_cells = ['b1c26', 'b2c28', 'b8c36',
                'b3c25', 'b2c14', 'b8c43', 
                'b2c18', 'b1c39', 'b3c4']

In [ ]:
# For full capacity prediction

fig = plt.figure(figsize=(18, 7))
         
for i, cell in enumerate(sample_cells):
    
    Q = test_raw[cell]['summary']['QDischarge']
    Q_eol = Q >= .88
    Q = Q[Q_eol]
    cl_Q = sg.medfilt(Q, 5)

    end = prediction_df.loc[cell]['EOL']
    x_Q = [1, prediction_df.loc[cell]['k-o'], prediction_df.loc[cell]['k-p'], end]
    y_Q = [cl_Q[0], prediction_df.loc[cell]['Qatk-o'], prediction_df.loc[cell]['Qatk-p'], cl_Q[-1]]

    end_ci = prediction_df.loc[cell]['EOL CI']
    ttko_ci = prediction_df.loc[cell]['k-o CI']
    ttkp_ci = prediction_df.loc[cell]['k-p CI']

    qko_ci = prediction_df.loc[cell]['Qatk-o CI']
    qkp_ci = prediction_df.loc[cell]['Qatk-p CI']

    x1 = np.arange(int(end_ci[0]))+1
    x2 =  np.arange(int(end_ci[1]))+1
    

    cb_Q = utils_models.modified_spline_evaluation(x_Q, y_Q, np.arange(int(end))+1)
    cb_Q_lw = utils_models.modified_spline_evaluation([1, ttko_ci[0], ttkp_ci[0], end_ci[0]], y_Q, x1)                                     
    cb_Q_up = utils_models.modified_spline_evaluation([1, ttko_ci[1], ttkp_ci[1], end_ci[1]], y_Q, x2)                                                 
    pts_Q = np.arange(len(cl_Q))+1
    
    ax = fig.add_subplot(3, 3, i+1)
    ax.text(0.02, 0.2, cell, transform=ax.transAxes, fontsize=16, fontweight='bold', va='top')
    ax.plot(pts_Q, cl_Q, 'k--', label='Actual curve', linewidth=1)
    ax.plot(np.arange(len(cb_Q))+1, cb_Q, color='brown', label='Predicted curve', linewidth=2.0)
    ax.fill(np.append(x1, x2[::-1]), np.append(cb_Q_lw, cb_Q_up[::-1]), color='brown', label=r'90% CI', alpha=0.13)
   
    
    if i==7:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, loc='upper center', ncol=3, fontsize=16, bbox_to_anchor=(0.5, -0.4))

    if i % 3 != 0:
        ax.set_yticklabels([])
    
    if i in [6, 7, 8]:
        ax.set_xlabel('Cycle', fontsize=16)

    ax.set_ylim([0.85, 1.1])




fig.text(0.09, 0.5, 'Capacity (Ah)', ha='center', va='center', rotation='vertical', fontsize=16)
plt.savefig(fname="plots/sig_level2_capacity-fade-curve.pdf", bbox_inches='tight')
#plt.savefig(fname="plots/sig_level2_capacity-fade-curve-subsample.pdf", bbox_inches='tight')


In [ ]:
# For full IR rise curve prediction
fig = plt.figure(figsize=(18, 7))

                          
for i, cell in enumerate(sample_cells):

    end = prediction_df.loc[cell]['EOL']
    end_ci = prediction_df.loc[cell]['EOL CI']

    x1 = np.arange(int(end_ci[0]))+1
    x2 =  np.arange(int(end_ci[1]))+1
    
    ir = test_raw[cell]['summary']['IR']
    cl_ir = sg.medfilt(ir, 5)
    
    x_ir = [1, prediction_df.loc[cell]['e-o'], prediction_df.loc[cell]['e-p'], end]
    y_ir = [cl_ir[0], prediction_df.loc[cell]['IRate-o'], prediction_df.loc[cell]['IRate-p'], prediction_df.loc[cell]['IRatEOL']]

    tteo_ci = prediction_df.loc[cell]['e-o CI']
    ttep_ci = prediction_df.loc[cell]['e-p CI']

    ireo_ci = prediction_df.loc[cell]['IRate-o CI']
    irep_ci = prediction_df.loc[cell]['IRate-p CI']
    ireol_ci = prediction_df.loc[cell]['IRatEOL CI']

    
    cb_ir = utils_models.modified_spline_evaluation(x_ir, y_ir, np.arange(int(end))+1)
    cb_ir_lw = utils_models.modified_spline_evaluation([1, tteo_ci[0], ttep_ci[0], end_ci[0]], y_ir, x1)                                         
    cb_ir_up = utils_models.modified_spline_evaluation([1, tteo_ci[1], ttep_ci[1], end_ci[1]], y_ir, x2)
    pts_ir = np.arange(len(cl_ir))+1
    
    ax = fig.add_subplot(3, 3, i+1)
    ax.text(0.05, 0.95, cell, transform=ax.transAxes, fontsize=16, fontweight='bold', va='top')

    ax.plot(pts_ir, cl_ir, 'k--', label='Actual curve', linewidth=1.0)
    ax.plot(np.arange(len(cb_ir))+1, cb_ir, color='brown', label='Predicted curve', linewidth=2.0)
    ax.fill(np.append(x1, x2[::-1]), np.append(cb_ir_lw, cb_ir_up[::-1]), color='brown', label=r'90% CI', alpha=0.13)
    
    if i==7:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, loc='upper center', ncol=3, fontsize=16, bbox_to_anchor=(0.5, -0.4))

    if i % 3 != 0:
        ax.set_yticklabels([])
    
    if i in [6, 7, 8]:
        ax.set_xlabel('Cycle', fontsize=16)

    ax.set_ylim([0.014, 0.022])


fig.text(0.08, 0.5, r'Internal Resistance ($\Omega$)', ha='center', va='center', rotation='vertical', fontsize=16)
plt.savefig(fname="plots/sig_level2_ir-rise-curve.pdf", bbox_inches='tight')
#plt.savefig(fname="plots/sig_level2_ir-rise-curve-subsample.pdf", bbox_inches='tight')



In [ ]:
# Load train true labels and corresponding predictions in order to produce parity plots
cycles_train_labels, cycles_train_pred = utils_gn.read_data('sig_cycles_train_labels.pkl', folder='models'), utils_gn.read_data('sig_cycles_train_pred.pkl', folder='models')
capir_train_labels, capir_train_pred = utils_gn.read_data('sig_capir_train_labels.pkl', folder='models'), utils_gn.read_data('sig_capir_train_pred.pkl', folder='models')

targets_to_plot = ['k-o', 'k-p', 'e-o', 'e-p', 'EOL', 'Qatk-o', 'Qatk-p', 'IRate-o', 'IRate-p', 'IRatEOL']
train_df = pd.DataFrame(columns=targets_to_plot)
train_pred_df = pd.DataFrame(columns=targets_to_plot)

for i, tg in enumerate(targets_to_plot[:5]):
    train_df[tg] = cycles_train_labels[:, i]
    train_pred_df[tg] = cycles_train_pred[:, i]

for i, tg in enumerate(targets_to_plot[5:]):
    train_df[tg] = capir_train_labels[:, i]
    train_pred_df[tg] = capir_train_pred[:, i]

test_df = utils_dgrd.create_knee_elbow_data(test_raw)

In [ ]:
fig = plt.figure(figsize=(14, 20))

for i, target in enumerate(targets_to_plot):


    ax = fig.add_subplot(5, 2, i+1)
    ax.text(0.05, 0.95, target, transform=ax.transAxes, fontsize=16, fontweight='bold', va='top')
    
    y_train_true = train_df[target].values
    y_train_pred = train_pred_df[target].values

    y_test_true = test_df[target].values
    y_test_pred = prediction_df[target].values

  
    ax.scatter(y_train_true, y_train_pred, s=50, color='royalblue', alpha=0.5, label='Train')
    ax.scatter(y_test_true, y_test_pred, s=50, color='brown', alpha=0.5, label='Test', marker='D')
    lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]

    # now plot both limits against each other
    ax.plot(lims, lims, 'k--', alpha=0.75, zorder=100)
    ax.set_xlim(lims)
    ax.set_ylim(lims)

    if i % 2 == 0:
        ax.set_ylabel('Predicted values', fontsize=16)
    
    if i in [8, 9]:
        ax.set_xlabel('Measured values', fontsize=16)
 
    
    
    #if i==5:
     #   handles, labels = ax.get_legend_handles_labels()
      #  ax.legend(handles, labels, loc='upper center', ncol=3, fontsize=16, bbox_to_anchor=(1.0, -0.2))
    
    # embed histogram of residuals
    subaxis = utils_models.add_sub_axes(ax, [0.62, 0.17, 0.35, 0.2])
    res_train = y_train_true - y_train_pred
    res_test = y_test_true - y_test_pred
    res = np.concatenate((res_train, res_test), casting='unsafe', dtype=float)
    subaxis.hist(res, bins=20, color='black', alpha=0.75, ec='black')
    subaxis.set_xlim(res.min(), -res.min())
    subaxis.set_xlabel('Residuals', fontsize=10)
    subaxis.set_ylabel('Frequency', fontsize=10)

    if i==8:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, loc='upper center', ncol=3, fontsize=16, bbox_to_anchor=(1.0, -0.2))

plt.savefig(fname="plots/sig_level2_parity-plot.pdf", bbox_inches='tight')
